In [1]:
import apache_beam as beam

/home/james/.local/lib/python3.5/site-packages/apache_beam/__init__.py:84: UserWarning: Running the Apache Beam SDK on Python 3 is not yet fully supported. You may encounter buggy behavior or missing features.
  'Running the Apache Beam SDK on Python 3 is not yet fully supported. '


In [28]:
def filter_on_count(element):
    name,count = element
    if count>30:
        return element
    
def format_output(element):
    name, count = element
    return ', '.join((name,str(count),'Regular employee'))
    

In [29]:
p1 = beam.Pipeline()
initial_collection =(
        p1
        | 'reading from a file'>>beam.io.ReadFromText('/home/james/beam/dept-data.txt')
        | 'spliting a row'>>beam.Map(lambda ele: ele.split(','))
)
account_count=(
        initial_collection
        | 'filtering only account dept'>>beam.Filter(lambda rcd: rcd[3] == 'Accounts')
        | 'mapping ac names'>>beam.Map(lambda x: ('Ac '+x[1],1))
        | 'counting ac names'>>beam.CombinePerKey(sum) #1
        | 'filtering on ac count'>> beam.Filter(filter_on_count) #2
        | 'output for ac'>>beam.Map(format_output) #3
        #| 'writing to a ac count file'>>beam.io.WriteToText('/home/james/beam/ac1')
   
   
)
hr_count=(
        initial_collection
        | 'filtering only hr dept'>>beam.Filter(lambda rcd: rcd[3] == 'HR')
        | 'mapping hr names'>>beam.Map(lambda x: ('HR '+x[1],1))
        | 'counting hr names'>>beam.CombinePerKey(sum) #1
        | 'filtering on hr count'>> beam.Filter(filter_on_count) #2
        | 'output for hr'>>beam.Map(format_output) #3
       # | 'writing to a hr count file'>>beam.io.WriteToText('/home/james/beam/hr1')
   
   
)
both_merge = (
        (account_count,hr_count)
        |beam.Flatten()
        | 'writing to a ac & hr count file'>>beam.io.WriteToText('/home/james/beam/both1')
)

p1.run()

In [30]:
! cat beam/both1-00000-of-00001

HR Leslie, 31, Regular employee
HR Vicky, 31, Regular employee
HR Beryl, 62, Regular employee
HR Olga, 31, Regular employee
HR Mindy, 31, Regular employee
HR Oscar, 31, Regular employee
HR Richard, 31, Regular employee
HR Kirk, 31, Regular employee
HR Kaori, 31, Regular employee
Ac Edouard, 31, Regular employee
Ac Rebekah, 31, Regular employee
Ac Gaston, 31, Regular employee
Ac Kumiko, 31, Regular employee
Ac Itoe, 31, Regular employee
Ac Marco, 31, Regular employee
Ac Kyle, 62, Regular employee


In [32]:
class my_transforms(beam.PTransform):
    def expand(self, input_cols):
        a = (
            input_cols
                | 'counting ac names'>>beam.CombinePerKey(sum) #1
                | 'filtering on ac count'>> beam.Filter(filter_on_count) #2
                | 'output for ac'>>beam.Map(format_output) #3
        )
        return a

In [33]:
p1 = beam.Pipeline()
initial_collection =(
        p1
        | 'reading from a file'>>beam.io.ReadFromText('/home/james/beam/dept-data.txt')
        | 'spliting a row'>>beam.Map(lambda ele: ele.split(','))
)
account_count=(
        initial_collection
        | 'filtering only account dept'>>beam.Filter(lambda rcd: rcd[3] == 'Accounts')
        | 'mapping ac names'>>beam.Map(lambda x: ('Ac '+x[1],1))
        | 'composite transform for ac'>>my_transforms()
        #| 'writing to a ac count file'>>beam.io.WriteToText('/home/james/beam/ac1')
   
   
)
hr_count=(
        initial_collection
        | 'filtering only hr dept'>>beam.Filter(lambda rcd: rcd[3] == 'HR')
        | 'mapping hr names'>>beam.Map(lambda x: ('HR '+x[1],1))
        | 'composite transform for hr'>>my_transforms()
       # | 'writing to a hr count file'>>beam.io.WriteToText('/home/james/beam/hr1')
   
   
)
both_merge = (
        (account_count,hr_count)
        |beam.Flatten()
        | 'writing to a ac & hr count file'>>beam.io.WriteToText('/home/james/beam/both1')
)

p1.run()

In [34]:
! cat beam/both1-00000-of-00001

HR Leslie, 31, Regular employee
HR Vicky, 31, Regular employee
HR Beryl, 62, Regular employee
HR Olga, 31, Regular employee
HR Mindy, 31, Regular employee
HR Oscar, 31, Regular employee
HR Richard, 31, Regular employee
HR Kirk, 31, Regular employee
HR Kaori, 31, Regular employee
Ac Edouard, 31, Regular employee
Ac Rebekah, 31, Regular employee
Ac Gaston, 31, Regular employee
Ac Kumiko, 31, Regular employee
Ac Itoe, 31, Regular employee
Ac Marco, 31, Regular employee
Ac Kyle, 62, Regular employee
